* Ссылки 
  * [Разведочный анализ данных](https://www.kaggle.com/emstrakhov/eda-with-pandas).<br>

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import seaborn as sns
from functools import reduce

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')

df = pd.read_csv('AmesHousing.txt', sep="\t", header = 0, encoding='cp1251', index_col=False)

In [ ]:
#Удаляем столбцы, где данные непрочитались
df = df.filter(regex='^(?!.*Unnamed).*$')

In [ ]:
#Удаляем строки с NaN значениями
df = df.loc[~df['Age'].isin(['NaN'])]
print('Текущая длина фрейма: ', len(df))

In [ ]:
#Если несколько классов, но хочется сделать классификацию строго бинарной, то разбиваем на группы ДА и НЕТ
df['date'] = df['date'].replace(0, 1)

In [ ]:
#Фильтр фрейма, оставить только строки с опредленным значением в одной из колонок
print( 'Before:', len(df) )
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]
df = df[df.Code.isin(gwa_codes)]
print( 'After:', len(df) )

In [ ]:
#Удаляем лишние колонки
columns = ['Name', 'Ticket', 'Cabin', 'Fare']
df.drop(columns, inplace=True, axis=1)

In [ ]:
# Удалим первые 100 пассажиров
df = df.drop(np.arange(100), axis=0)

In [ ]:
# Удалим всех пацентов с возрастом меньше 10 и старше 50 лет
df1 = df.drop(df[(df['Age'] < 10) | (df['Age'] > 50)].index)
df1.shape[0] / df.shape[0]

In [ ]:
#Перекодировка в булени
df['Survived'] = df['Survived'].astype(bool)

In [ ]:
#Повернем фрейм, разбив один столбец на несколько, в зависимости от значений, все объекты выровнены по дате
pivoted_df = df.pivot(index='Date', columns='Code', values='VWAP')
pivoted_df.tail()